In [1]:
cd cmput_651_final/

/home/zeyi/work_station/gpu/cmput_651_final


In [2]:
%run init.ipynb

In [3]:
selected_worlds = get_selected()
world_name = 'den020d.map'
world = selected_worlds[world_name]

maps/selected
maps/selected/selected-map/lak110d.map
maps/selected/selected-map/den020d.map
maps/selected/selected-map/den005d.map
maps/selected/selected-map/ost102d.map
maps/selected/selected-map/den001d.map



In [4]:
roads = list(filter(lambda t: world.get(t) == World.ROAD, world))
idx_to_pos = dict(enumerate(roads))
vocab_size = len(idx_to_pos)
pos_to_idx = {v: k for k, v in idx_to_pos.items()}
print('vocab_size:', vocab_size)

vocab_size: 3102


In [5]:
def encode(pos):
    vec = np.zeros(vocab_size, dtype=np.int64)
    vec[pos_to_idx[pos]] = 1
    return vec

def decode(vec):
    return vec.argmax()

In [20]:
stream_dataset = StreamingPointsDataset(size=256, sample_rate=1)
batch_size = 1024 * 36

point_data_loader = torch.utils.data.DataLoader(
    stream_dataset,
    batch_size=batch_size,
    num_workers=8,
)

In [17]:
embed_dim = 64
hidden_dim = 1024
model = LargeEmbedModel(vocab_size=vocab_size, embed_dim=embed_dim, hidden_dim=hidden_dim)
model.to(device)

LargeEmbedModel(
  (embed): Embedding(3102, 64)
  (fc1): Linear(in_features=128, out_features=1024, bias=True)
  (do1): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=1024, out_features=3102, bias=True)
)

In [18]:
optimizer = optim.Adam(model.parameters(), weight_decay=1e-5, lr=1e-3)
# criterion = F.mse_loss
criterion = F.nll_loss

In [21]:
model.train()
model.to(device)
for epoch in tqdm_nb(range(100)):
    losses = []
    for batch_idx, (x, y) in enumerate(point_data_loader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        output = model(x)
#         print(output.shape)
#         print(y.shape)
        loss = criterion(output, y.reshape(-1))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    torch.save(model, f'{world_name}_{str(model.__class__.__name__)}_2.pt')
    print('epoch: ', epoch,
          '\tloss: ',
          '\t', np.sum(np.array(losses)),
          '\t', np.mean(np.array(losses)),
#           '\t', np.mean(np.array(losses)) / batch_size,
#           '\t', np.mean(np.array(losses)) / len(point_data_loader),
         )
model.to('cpu')

epoch:  0 	loss:  	 7.4514442682266235 	 0.9314305335283279
epoch:  1 	loss:  	 6.543390989303589 	 0.8179238736629486
epoch:  2 	loss:  	 6.835068941116333 	 0.8543836176395416
epoch:  3 	loss:  	 6.7241411209106445 	 0.8405176401138306
epoch:  4 	loss:  	 6.654769241809845 	 0.8318461552262306
epoch:  5 	loss:  	 6.460136830806732 	 0.8075171038508415
epoch:  6 	loss:  	 6.791754961013794 	 0.8489693701267242
epoch:  7 	loss:  	 6.346808075904846 	 0.7933510094881058
epoch:  8 	loss:  	 6.540900886058807 	 0.8176126107573509
epoch:  9 	loss:  	 6.8741759061813354 	 0.8592719882726669
epoch:  10 	loss:  	 6.4907044768333435 	 0.8113380596041679
epoch:  11 	loss:  	 6.88770318031311 	 0.8609628975391388
epoch:  12 	loss:  	 6.133929193019867 	 0.7667411491274834
epoch:  13 	loss:  	 6.379638314247131 	 0.7974547892808914
epoch:  14 	loss:  	 5.868942320346832 	 0.733617790043354
epoch:  15 	loss:  	 5.980635941028595 	 0.7475794926285744
epoch:  16 	loss:  	 6.1071677803993225 	 0.7633

LargeEmbedModel(
  (embed): Embedding(3102, 64)
  (fc1): Linear(in_features=128, out_features=1024, bias=True)
  (do1): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=1024, out_features=3102, bias=True)
)

In [ ]:
torch.save(model, f'{world_name}_{str(model.__class__.__name__)}.pt')